In [ ]:
!wget https://media.githubusercontent.com/media/hunarbatra/covid-spike-data/master/spike-gisaid-50k.txt

In [ ]:
data = []

In [ ]:
with open('spike-gisaid-50k.txt', 'r') as file:
    for line in file:
        data.append(line)

In [ ]:
data = data[:10000]

In [ ]:
with open('validation-set.txt', 'w') as file:
    for line in data:
        file.write(line) 

In [ ]:
!pip install git+https://github.com/huggingface/transformers

In [ ]:
from transformers import pipeline

mask_filler = pipeline(
    "fill-mask", 
    model="yarongef/DistilProtBert",
    tokenizer="yarongef/DistilProtBert"
)

Downloading:   0%|          | 0.00/589 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/924M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
mask_filler("MAT[MASK]ALM")

[{'score': 0.10112512111663818, 'token': 5, 'token_str': 'L', 'sequence': 'L'},
 {'score': 0.07724442332983017,
  'token': 10,
  'token_str': 'S',
  'sequence': 'S'},
 {'score': 0.07407709956169128,
  'token': 13,
  'token_str': 'R',
  'sequence': 'R'},
 {'score': 0.06858670711517334,
  'token': 11,
  'token_str': 'I',
  'sequence': 'I'},
 {'score': 0.06788940727710724,
  'token': 12,
  'token_str': 'K',
  'sequence': 'K'}]

# Variant Mutation Processing

In [ ]:
import pandas as pd
import math
from statistics import mean

In [ ]:
def isNaN(num):
    return num != num

In [ ]:
mutations = ['D614G', 'D950N', 'E484K', 'H655Y', 'H655Y', 'K417N', 'N501Y', 'P681H', 'Q493R']

# Common Variant Mutations Masking Testing

In [ ]:
mutations_seqs = {}
[mutations_seqs.setdefault(name, []) for name in mutations]

mutations_seqs_results = {}
[mutations_seqs_results.setdefault(name, []) for name in mutations]

mutations_seqs_original = {}
[mutations_seqs_original.setdefault(name, []) for name in mutations]

[[], [], [], [], [], [], [], [], []]

In [ ]:
line_count = 0

with open('validation-set.txt', 'r') as file:
    for line in file:
        line_count += 1
        print(line_count)
        for mutation in mutations:
            original = mutation[0]
            change = mutation[-1]
            position = int("".join(mutation[1:-1]))-1

            if position < len(line) and line[position] == original:
                mutations_seqs[mutation].append(line)
                sequence = line[:position] + '[MASK]' + line[position+1:]
                curr_results = mask_filler(sequence)

                curr_sequence_pred = 0
                curr_sequence_nochange = 0

                for i in range(0, len(curr_results)):
                    if curr_results[i]['token_str'] == change:
                        curr_sequence_pred = curr_results[i]['score']
                    elif curr_results[i]['token_str'] == original:
                        curr_sequence_nochange = curr_results[i]['score']
                    if curr_sequence_pred > 0 and curr_sequence_nochange > 0:
                        break
                    
                mutations_seqs_results[mutation].append(curr_sequence_pred)
                mutations_seqs_original[mutation].append(curr_sequence_nochange)

In [ ]:
mutations_seqs_results_mean = {}
[mutations_seqs_results_mean.setdefault(name, 0) for name in mutations]

mutations_seqs_original_mean = {}
[mutations_seqs_original_mean.setdefault(name, 0) for name in mutations]

for mutation in mutations_seqs_original:
    curr_list = mutations_seqs_original[mutation]
    mutations_seqs_original_mean[mutation] = round(mean(curr_list), 2)

for mutation in mutations_seqs_results:
    curr_list = mutations_seqs_results[mutation]
    mutations_seqs_results_mean[mutation] = round(mean(curr_list), 2)

In [ ]:
mutations_seqs_results_mean

{'D614G': 0,
 'D950N': 0,
 'E484K': 0.07,
 'H655Y': 0,
 'K417N': 0,
 'N501Y': 0,
 'P681H': 0,
 'Q493R': 0.08}

In [ ]:
mutations_seqs_original_mean

{'D614G': 0,
 'D950N': 0,
 'E484K': 0,
 'H655Y': 0,
 'K417N': 0.07,
 'N501Y': 0,
 'P681H': 0,
 'Q493R': 0}

In [ ]:
for mutation in mutations_seqs_results:
    print(len(mutations_seqs_results[mutation]))

68
2063
1799
3776
2012
1878
1708
2171


In [ ]:
for mutation in mutations_seqs_results:
    print(mutation + ' : ' + str(max(mutations_seqs_results[mutation])))

D614G : 0
D950N : 0
E484K : 0.06908921152353287
H655Y : 0
K417N : 0
N501Y : 0
P681H : 0
Q493R : 0.07632523775100708
